# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [100]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [101]:
df = pd.read_csv('output_data/cities.csv')
df

,Unnamed: 0,City,Lat,Lng,Temp,Humidity,Cloudiness,Wind,Country,Date
0,0,hilo,19.7297,-155.0900,75.20,60,1,4.61,US,1612293749
1,1,carnarvon,-24.8667,113.6333,75.20,100,40,21.85,AU,1612293830
2,2,punta arenas,-53.1500,-70.9167,69.80,49,40,17.00,CL,1612293567
3,3,ushuaia,-54.8000,-68.3000,53.60,82,75,14.97,AR,1612293808
4,4,sobradinho,-12.8333,-39.1000,84.02,63,55,9.44,BR,1612293830
...,...,...,...,...,...,...,...,...,...,...
587,587,ramjibanpur,22.8333,87.6167,55.22,43,0,3.40,IN,1612293902
588,588,mansa,29.9833,75.3833,56.46,39,0,6.08,IN,1612293902
589,589,ipixuna,-1.7625,-48.8043,89.60,58,75,10.36,BR,1612293902
590,590,san juan,-31.5375,-68.5364,73.40,33,93,11.59,AR,1612293769


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [102]:
#grab locations by lat and long
locations = df[["Lat", "Lng"]]

#grab humidity column for weight
weight = df['Humidity']

In [103]:
#plot heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weight,
                                 dissipating=False, max_intensity=300,
                                 point_radius=5)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [104]:
narrow_cities =  df.loc[(df["Humidity"] < 80) & (df["Humidity"] > 10) & (df["Cloudiness"] == 0) & (df["Temp"] > 75.00) & (df["Temp"] < 80.00) & (df["Wind"] < 10.00)].reset_index(drop=True)
del narrow_cities["Unnamed: 0"]
narrow_cities

,City,Lat,Lng,Temp,Humidity,Cloudiness,Wind,Country,Date
0,coihaique,-45.5752,-72.0662,75.20,43,0,4.61,CL,1612293831
1,madimba,-10.4000,40.3333,78.94,76,0,5.01,TZ,1612293833
2,pisco,-13.7000,-76.2167,77.00,65,0,8.05,PE,1612293870
3,mecca,21.4267,39.8261,75.45,73,0,1.97,SA,1612293843
4,calvinia,-31.4707,19.7760,76.71,31,0,8.23,ZA,1612293883


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [105]:
hotel_df = narrow_cities
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Temp,Humidity,Cloudiness,Wind,Country,Date,Hotel Name
0,coihaique,-45.5752,-72.0662,75.20,43,0,4.61,CL,1612293831,
1,madimba,-10.4000,40.3333,78.94,76,0,5.01,TZ,1612293833,
2,pisco,-13.7000,-76.2167,77.00,65,0,8.05,PE,1612293870,
3,mecca,21.4267,39.8261,75.45,73,0,1.97,SA,1612293843,
4,calvinia,-31.4707,19.7760,76.71,31,0,8.23,ZA,1612293883,


In [106]:
# create a params dict that will be updated with new city each iteration
# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "lodging",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
print(hotel_df["Hotel Name"])

0                    Diego De Almagro Coyhaique
1                                    Til Hotels
2    DoubleTree Resort by Hilton Paracas - Peru
3    Makkah Clock Royal Tower, A Fairmont Hotel
4                                Die Lemoenhuis
Name: Hotel Name, dtype: object


In [107]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [110]:
# Create a marker_layer using the hotel name list to fill the info box
fig = gmaps.figure()
markers = gmaps.marker_layer(locations,
    info_box_content=[f" {name}" for name in hotel_info])
fig.add_layer(markers)
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))